# input length test

> input length 512 이하의 행이 몇 개나 있는지?

In [ ]:

import sys
import os

from transformers import AutoTokenizer, BartForConditionalGeneration
import json
import pandas as pd

model_name = "alaggung/bart-r3f"
max_length = 512
num_beams = 10
length_penalty = 1.2

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='../cache')
model = BartForConditionalGeneration.from_pretrained(model_name, cache_dir='../cache')
model.eval()

# 'inputs', 'labels', 'preds', 'rouge', 'bleu'
df = pd.DataFrame()

def make_data(path):
    inputs = []
    labels = []

    with open(path, "r") as f:
        data = json.load(f)

    def make_chat(inp):
        for cvt in inp['conversation']:
            chats = []
            speaker = cvt['speaker']
            utterance = cvt['utterance']
            chats.append(f"화자{speaker}: {utterance}")
        
        chat = "[BOS]" + "[SEP]".join(chats)  + "[EOS]"
        return chat

    for example in data:
        chat = make_chat(example["input"])
        
        input = tokenizer(chat, return_tensors="pt")
        if input.input_ids.size(1) > 512:
            continue

        inputs.append(input)
        labels.append(example["output"])

    return inputs, labels

inputs, labels = make_data("/mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/resource/data/train.json")

In [ ]:
len(inputs)

# Hanspell 성능 테스트

대화체에서도 맞춤법 교정이 유효한가?

In [1]:
%cd /mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/py-hanspell

/mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/py-hanspell


/home/hwyewon/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import re
import requests

def get_passport_key():
    """네이버에서 '네이버 맞춤법 검사기' 페이지에서 passportKey를 획득

        - 네이버에서 '네이버 맞춤법 검사기'를 띄운 후 
        html에서 passportKey를 검색하면 값을 찾을 수 있다.

        - 찾은 값을 spell_checker.py 48 line에 적용한다.
    """

    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=네이버+맞춤법+검사기"
    res = requests.get(url)

    html_text = res.text

    match = re.search(r'passportKey=([^&"}]+)', html_text)
    if match:
        passport_key = match.group(1)
        return passport_key
    else:
        return False


def fix_spell_checker_py_code(file_path, passportKey):
    print(passportKey)
    """획득한 passportkey를 spell_checker.py파일에 적용
    """
    
    pattern = r"'passportKey': '.*'"

    with open(file_path, 'r', encoding='utf-8') as input_file:
        content = input_file.read()
        modified_content = re.sub(pattern, f"'passportKey': '{passportKey}'", content)

    with open(file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(modified_content)
    
    return 


# before run
spell_checker_file_path = './hanspell/spell_checker.py'

passport_key = get_passport_key()
if passport_key:
    fix_spell_checker_py_code(spell_checker_file_path, passport_key)
else:
    print("passportKey를 찾을 수 없습니다.")

7f09f275f8f4fea55bf2d825b99a2f58ea887506


In [4]:
from hanspell import spell_checker

result = spell_checker.check(u'나도 인제 안 미루고 맞아는 보려고. 맞아서 소용없으면 소용없는 거고 또 맞아서 좋을 수도 있는 거니까. 그래서 안 걸리면 더 좋은 거고. 가 한번 name3이가 얘기한 대로 혜택 받을 수 있으면 받아가지고 맞아보고.')
result

Checked(result=True, original='나도 인제 안 미루고 맞아는 보려고. 맞아서 소용없으면 소용없는 거고 또 맞아서 좋을 수도 있는 거니까. 그래서 안 걸리면 더 좋은 거고. 가 한번 name3이가 얘기한 대로 혜택 받을 수 있으면 받아가지고 맞아보고.', checked='나도 인제 안 미루고 맞아는 보려고. 맞아서 소용없으면 소용없는 거고 또 맞아서 좋을 수도 있는 거니까. 그래서 안 걸리면 더 좋은 거고. 가 한번 name3이가 얘기한 대로 혜택받을 수 있으면 받아가지고 맞아보고.', errors=1, words=OrderedDict([('나도', 0), ('인제', 0), ('안', 0), ('미루고', 0), ('맞아는', 0), ('보려고.', 0), ('맞아서', 0), ('소용없으면', 0), ('소용없는', 0), ('거고', 0), ('또', 0), ('좋을', 0), ('수도', 0), ('있는', 0), ('거니까.', 0), ('그래서', 0), ('걸리면', 0), ('더', 0), ('좋은', 0), ('거고.', 0), ('가', 0), ('한번', 0), ('name3이가', 0), ('얘기한', 0), ('대로', 0), ('혜택받을', 2), ('수', 0), ('있으면', 0), ('받아가지고', 0), ('맞아보고.', 0)]), time=0.07621264457702637)

In [ ]:
result.checked

In [ ]:
result.original

# 반복 어구 없애기

In [ ]:
import re

# 테스트 문자열 리스트
test_strings = [
    "name1 is John",
    "name123",
    "My name1 is Jane",
    "name_underscore",
    "123name",
    "너는 유산균 아직 안 먹였으면 한번 알아봐가지고 특히 name1이 먼저 먹여. name1이가 밥도 잘 안 먹고 하니까 내가 추천해 주는 거 이로울 만한 거는 없는데 한번 알아봐봐. 있을 거야"
]

# 정규표현식 패턴r'\bname\S*'
pattern = r'name[0-9]\S*'

# 정규표현식을 사용하여 어구 찾기
matches = [re.findall(pattern, string) for string in test_strings]

# 결과 출력
for string, match in zip(test_strings, matches):
    print(f"Original string: {string}")
    print(f"Matches: {match}\n")

# Hanspell 제외 전처리 테스트

In [ ]:
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import koreanize_matplotlib

def make_data(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)

    text_list = []
    def make_chat(inp):
        chat = ""
        for cvt in inp['conversation']:
            chat += cvt['utterance']
        return chat

    for example in data:
        text_list.append(make_chat(example['input']))
    
    text = ' '.join(text_list)
    return text



FONT_PATH = "/mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/NanumSquareNeo-cBd.ttf"

def make_cloud(text, title, top_k, max_words=100):
    # plt.rc('font', family='NanumGothicCoding')
    wordcloud = WordCloud(font_path=FONT_PATH, max_words=max_words, background_color="white").generate(text)
    
    fig, axs = plt.subplots(2, 1, figsize=(15, 15))

    top_data = dict(list(wordcloud.words_.items())[:top_k])
    labels, values = zip(*top_data.items())

    # 막대 그래프 그리기
    axs[0].bar(labels, values)
    axs[0].set_xlabel('Labels')
    axs[0].set_ylabel('Values')
    axs[0].set_title(f"max count words on {title} data (top {top_k})", fontsize=36)


    # 두 번째 그래프: 워드클라우드
    axs[1].imshow(wordcloud)
    axs[1].axis('off')
    axs[1].set_title(f"max count words on {title} data (max words = {max_words})", fontsize=36)

    # 그래프 간격 조정
    plt.tight_layout()
    plt.show()

In [ ]:
import re
import json

## Preprocess functions ##
def remove_empty_utterance(data:json):
    """
    Remove empty utterances from the data
    """
    for example in data:
        example['input']['conversation'] = [cvt for cvt in example['input']['conversation'] if cvt['utterance'] != '']
    return data


def correct_wrong_output(data:json):
    """
    Correct wrong speakers in outputs of train samples 'train-000401', 'train-000402'
    """
    data[400]['output'] = data[400]['output'].replace('SD2100504','SD2110504')
    data[401]['output'] = data[401]['output'].replace('SD2110503','SD2100503')

    return data


def file_preprocess(data:json):
    data = remove_empty_utterance(data)
    data = correct_wrong_output(data)

    return data


"""
불용어 처리
- name1, name2..
- 뒤에 물결이 붙는 경우 ("음~", "아~")
- 그, 뭐, 어, 인제, 막, 아, 음, 읍, 오, 으
- 한 글자가 두번 이상 반복되는 경우 ("또 또", "그 그")
"""

stopwords_pattern = [r'name[0-9]\S*', r'\w~', r'\b으\b', r'\b그\b', r'\b뭐\b', r'\b어\b',  r'\b인제\b', r'\b막\b', r'\b아\b', r'\b음\b', r'\b읍\b', r'\b오\b', r'\b으\b']

def remove_stopwords(text):
    for pattern in stopwords_pattern:
        text = re.sub(pattern, '', text)
    
    # 두 번 이상 반복되는 경우
    text = re.sub(r'\b(\w)\s+\1\b', r'\1', text)
    
    return text


# stopwords + 반복어구 제거
def text_preprocess(text):
    text = remove_stopwords(text)
    return text

In [ ]:
import json
from tqdm import tqdm

def make_clean_data(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)

    data = file_preprocess(data)
    
    text_list = []
    def make_chat(inp):
        chat = ""
        for cvt in inp['conversation']:
            chat += cvt['utterance']

        return text_preprocess(chat)

    for example in tqdm(data):
        text_list.append(make_chat(example['input']))
    
    text = ' '.join(text_list)
    return text

In [2]:
import re
re.sub(r'name[0-9]\S*', '', "name1이가 그래서 정말 슬펐어")

' 그래서 정말 슬펐어'

In [3]:
from datetime import datetime

# 현재 날짜와 시간 가져오기
now = datetime.now()

# 날짜와 시간을 문자열로 포맷팅
current_time = now.strftime("%Y-%m-%d %H:%M")

print(current_time)

2024-07-22 04:04


# 화자 1 - 화자 2

꼭 넣어줘야 할까?

# BART 요약 테스트

요약해서 input length를 줄인다면?